### Single Stock Analysis Using 3 Factor Risk Model

In [78]:
import datetime as dt
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

tickers = ['IBM']
start = "2000-01-01"
end = "2020-01-01"

# Grab yfinance Data
df_portfolio = yf.download(tickers, start, end).reset_index()

df_portfolio = df_portfolio[["Date", "Adj Close"]].rename(columns={"Date": "date", "Adj Close": "adj_close"})


[*********************100%***********************]  1 of 1 completed


In [79]:
# Computing Nominal Range
data = df_portfolio["adj_close"]
data/data.iloc[0]

0       1.000000
1       0.966056
2       1.000000
3       0.982759
4       0.978448
          ...   
5026    1.736327
5027    1.735427
5028    1.740058
5029    1.708413
5030    1.724235
Name: adj_close, Length: 5031, dtype: float64

In [86]:
# Calculate Log Returns
df_portfolio["return"] = (df_portfolio["adj_close"] /df_portfolio["adj_close"].shift())

df_portfolio["log_return"] = np.log(df_portfolio["return"])

# Total Return Over Time Series
np.e ** df_portfolio["log_return"].sum()

1.7242353484717037

In [103]:
import pandas as pd

# Calculating Stocks Monthly Returns
df_portfolio_monthly  = df_portfolio[["date", "adj_close"]].set_index("date").resample("M").last().pct_change().dropna()

# Load in Fama French Data
df_fama_french = pd.read_csv("data\\Developed_3_Factors.csv")[:390]

# Clean Fama French Data
df_fama_french = df_fama_french.rename(columns={"Unnamed: 0": "date", "Mkt-Rf": "market_premium", "SMB": "smb", "HML": "hml", "RF": "rf"})

df_fama_french

,date,Mkt-RF,SMB,HML,RF
0,199007,0.77,0.53,-0.36,0.68
1,199008,-10.77,-1.51,0.48,0.66
2,199009,-11.89,1.32,0.81,0.60
3,199010,9.35,-7.58,-4.48,0.68
4,199011,-3.72,1.44,1.13,0.57
...,...,...,...,...,...
385,202208,-4.22,0.42,2.49,0.19
386,202209,-9.46,-1.66,1.89,0.19
387,202210,6.79,-2.02,4.41,0.23
388,202211,7.25,-0.24,-0.28,0.29
